In [1]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
import sqlalchemy
# from Keys import MySQL_pass
# from Keys import OW_API_key

## Read the "cities" table from the database into the notebook

1. With the OpenWeather API we need to use the name of the city in the url. We will use the "cities" table from our database to loop over the "City_name" column and thus implement them (one at a time) in the url.

2. Since the weather dataframe (wich will contain the data we will get from the openweather API) will be related to the "cities" table in the database we need to get the corresponding city id from the cities table and implement it in the weather one (will become a foreign key here). This will be done during the for loop from above.

In [2]:
schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = "Bender1999!!" # Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
cities_df = pd.read_sql("cities", con=connection_string)
cities_df

,City_id,City_name,Country
0,1,Berlin,Germany
1,2,Hamburg,Germany
2,3,Munich,Germany


## Create a weather dataframe with info for all cities in the database

In [ ]:
berlin_timezone = timezone('Europe/Berlin')
API_key = OW_API_key # USE YOUR OPENWEATHER API KEY
weather_items = []

In [ ]:
# For loop over the "City_name" column in the "cities_df"

for i in cities_df["City_name"]:
    print(i)

Berlin
Hamburg
Munich


In [ ]:
# Extracting the "City_id" from the cities table. This will be implemented in the for loop.
cities_df.loc[cities_df["City_name"] == "Hamburg", "City_id"].values[0]

2

In [ ]:
# Slightly different way to extract the "City_id"
#cities_df.loc[cities_df["City_name"] == "Hamburg", "City_id"].to_list()[0]

2

In [ ]:
for city in cities_df["City_name"]:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()
    # As we are now using the data from our relational database
    # the city should reflect the city_id and not the city name
    city_id = cities_df.loc[cities_df["City_name"] == city, "City_id"].values[0]

    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for item in json["list"]:
        weather_item = {
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "outlook": item["weather"][0].get("description", None),
            "temperature": item["main"].get("temp", None),
            "feels_like": item["main"].get("feels_like", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "rain_prob": item.get("pop", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)

In [ ]:
weather_df = pd.DataFrame(weather_items)

In [ ]:
weather_df

,city_id,forecast_time,outlook,temperature,feels_like,rain_in_last_3h,wind_speed,rain_prob,data_retrieved_at
0,1,2024-06-06 15:00:00,clear sky,21.15,20.47,0.00,3.79,0.0,2024-06-06 15:57:04
1,1,2024-06-06 18:00:00,scattered clouds,20.10,19.34,0.00,3.14,0.0,2024-06-06 15:57:04
2,1,2024-06-06 21:00:00,broken clouds,18.12,17.24,0.00,1.69,0.0,2024-06-06 15:57:04
3,1,2024-06-07 00:00:00,light rain,15.03,14.13,0.15,0.52,0.2,2024-06-06 15:57:04
4,1,2024-06-07 03:00:00,scattered clouds,13.42,12.51,0.00,1.64,0.0,2024-06-06 15:57:04
...,...,...,...,...,...,...,...,...,...
115,3,2024-06-11 00:00:00,overcast clouds,11.58,11.30,0.00,1.76,0.0,2024-06-06 15:57:04
116,3,2024-06-11 03:00:00,light rain,11.79,11.61,2.02,2.15,1.0,2024-06-06 15:57:04
117,3,2024-06-11 06:00:00,light rain,12.38,12.18,0.15,1.84,1.0,2024-06-06 15:57:04
118,3,2024-06-11 09:00:00,light rain,12.73,12.57,2.07,1.57,1.0,2024-06-06 15:57:04


In [ ]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city_id            120 non-null    int64  
 1   forecast_time      120 non-null    object 
 2   outlook            120 non-null    object 
 3   temperature        120 non-null    float64
 4   feels_like         120 non-null    float64
 5   rain_in_last_3h    120 non-null    float64
 6   wind_speed         120 non-null    float64
 7   rain_prob          120 non-null    float64
 8   data_retrieved_at  120 non-null    object 
dtypes: float64(5), int64(1), object(3)
memory usage: 8.6+ KB


In [ ]:
weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

In [ ]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   city_id            120 non-null    int64         
 1   forecast_time      120 non-null    datetime64[ns]
 2   outlook            120 non-null    object        
 3   temperature        120 non-null    float64       
 4   feels_like         120 non-null    float64       
 5   rain_in_last_3h    120 non-null    float64       
 6   wind_speed         120 non-null    float64       
 7   rain_prob          120 non-null    float64       
 8   data_retrieved_at  120 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(1), object(1)
memory usage: 8.6+ KB


## # Create "weather" table in the database

```sql
CREATE TABLE weather (
	weather_id INT AUTO_INCREMENT,
    city_id INT NOT NULL,
    forecast_time DATETIME,
    outlook VARCHAR(255),
    temperature FLOAT,
    feels_like FLOAT,
    rain_in_last_3h FLOAT,
    wind_speed FLOAT,
    rain_prob FLOAT,
    data_retrieved_at DATETIME,
    PRIMARY KEY (weather_id),
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
);
```

In [7]:
names = {"number":["1","2","3"],"name":["guy","fried","piki"]}

In [8]:
names_df=pd.DataFrame(names)

## Push the "weather_df" to the empty "weather" table in the database

In [12]:
! pip install pymysql



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.5 MB/s eta 0:00:00


In [13]:
names_df.to_sql('mynames',
                  if_exists='append',
                  con=connection_string,
                  index=False)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

# Making our function work with our SQL cities data
This section delves into a more advanced aspect of the project, which may be beyond the point many of you got to. However, it serves as an illustrative example of how to structure your work effectively. It demonstrates how we can compartmentalise various aspects of our data processing pipeline into distinct functions, enhancing code readability and facilitating debugging. It's a good idea to start writing your pipeline in a similar manner ready for the cloud.
> This part will only work locally, not on colab. You must also create your weather table in SQL first.

In [ ]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from Keys import MySQL_pass
from Keys import OW_API_key

def retreiving_and_sending_data():
  connection_string = connection()
  cities_df = get_cities_data(connection_string)
  weather_df = get_weather_data(cities_df)
  send_weather_data(weather_df, connection_string)
  return "Data has been updated"

def connection():
  schema = "gans_local" # The name of your database
  host = "127.0.0.1"
  user = "root"
  password = MySQL_pass # Your MySQL password
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def get_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

def get_weather_data(cities_df):
  berlin_timezone = timezone('Europe/Berlin')
  API_key = OW_API_key # USE YOUR OPENWEATHER API KEY
  weather_items = []

  for city in cities_df["City_name"]:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    # As we are now using the data from our relational database
    # the city should reflect the city_id and not the city name
    city_id = cities_df.loc[cities_df["City_name"] == city, "City_id"].values[0]

    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for item in json["list"]:
        weather_item = {
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "outlook": item["weather"][0].get("description", None),
            "temperature": item["main"].get("temp", None),
            "feels_like": item["main"].get("feels_like", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "rain_prob": item.get("pop", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

  return weather_df

def send_weather_data(weather_df, connection_string):
  weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)

In [ ]:
retreiving_and_sending_data()

'Data has been updated'

In [ ]:
pd.read_sql("weather", con=connection_string)

,city_id,forecast_time,outlook,temperature,feels_like,rain_in_last_3h,wind_speed,rain_prob,data_retrieved_at
0,1,2024-06-06 15:00:00,clear sky,21.38,20.72,0.00,3.79,0.0,2024-06-06 16:17:04
1,1,2024-06-06 18:00:00,scattered clouds,20.25,19.51,0.00,3.14,0.0,2024-06-06 16:17:04
2,1,2024-06-06 21:00:00,broken clouds,18.19,17.32,0.00,1.69,0.0,2024-06-06 16:17:04
3,1,2024-06-07 00:00:00,light rain,15.03,14.13,0.15,0.52,0.2,2024-06-06 16:17:04
4,1,2024-06-07 03:00:00,scattered clouds,13.42,12.51,0.00,1.64,0.0,2024-06-06 16:17:04
...,...,...,...,...,...,...,...,...,...
115,3,2024-06-11 00:00:00,overcast clouds,11.58,11.30,0.00,1.76,0.0,2024-06-06 16:17:04
116,3,2024-06-11 03:00:00,light rain,11.79,11.61,2.02,2.15,1.0,2024-06-06 16:17:04
117,3,2024-06-11 06:00:00,light rain,12.38,12.18,0.15,1.84,1.0,2024-06-06 16:17:04
118,3,2024-06-11 09:00:00,light rain,12.73,12.57,2.07,1.57,1.0,2024-06-06 16:17:04
